<!-- LTeX: language=fr -->

Cours 5 : consommer des API web
================================

**Loïc Grobol** [<lgrobol@parisnanterre.fr>](mailto:lgrobol@parisnanterre.fr)


In [1]:
%pip install -U requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.3 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.12.14-py3-none-any.whl (164 kB)
Using cached charset_normalizer-3.4.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (145 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.


Ce cours est partiellement adapté du tutoriel [Python and REST
APIs](https://realpython.com/api-integration-in-python/) de Real Python.

## API ?

***A**pplication **P**rogramming **I**nterface*, en français parfois « interface de programmation
d’applications » mais surtout API \[eɪpiˈaɪ\]. À ne pas confondre avec
l'[API](https://www.internationalphoneticalphabet.org) des phonéticiens (puisqu'en anglais, c'est
l'IPA, à ne pas confondre avec les bières enrichies en houblon \[vous suivez ?\]).

Il s'agit d'*interfaces* de communications entre *applications*. À la différence des interfaces
humain⋅es – machines (même si les deux classes ne sont pas disjointes, d'ailleurs est-ce que vous
voyez des exemples qui sont les deux ?). Autrement dit, une API c'est la surface d'une application,
son panneau de commande accessible par d'autres applications. On suppose en général que ces
interfaces sont

- Publiques
- Documentées
- Stables
- Opaques

Le dernier point, l'*opacité* rejoint les considérations de séparation des préoccupations qu'on a
déjà abordées plusieurs fois : quand j'accède à une application via son API, je ne veux pas avoir à
me soucier de ce qui se passe en interne. Tout ce qui compte pour moi, c'est ce que j'y mets et ce
que j'en récupère.

Point vocabulaire : si une application A utilise l'API d'une application B, on dira que A est le
*client* et B le *serveur*.

## APIs web

Une API web, c'est une API à laquelle on accède via le web.

Voilà, le cours est fini, joyeux Noël.

En pratique, on parle d'API web quand des services applicatifs sont accessibles via des requêtes
HTTP. Par exemple celle de GitHub, à laquelle on accède à <https://api.github.com>. Regardez ce qui
se passe par exemple avec une requête `GET` sur le point d'accès (*endpoint*)
<https://api.github.com/users/loicgrobol>

In [21]:
!curl https://api.github.com/users/loicgrobol

/bin/bash: ligne 1: curl : commande introuvable


In [5]:
%pip install httpx

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached anyio-4.8.0-py3-none-any.whl (96 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import httpx
print(httpx.get("https://api.github.com/users/loicgrobol").text)

{"login":"LoicGrobol","id":14248012,"node_id":"MDQ6VXNlcjE0MjQ4MDEy","avatar_url":"https://avatars.githubusercontent.com/u/14248012?v=4","gravatar_id":"","url":"https://api.github.com/users/LoicGrobol","html_url":"https://github.com/LoicGrobol","followers_url":"https://api.github.com/users/LoicGrobol/followers","following_url":"https://api.github.com/users/LoicGrobol/following{/other_user}","gists_url":"https://api.github.com/users/LoicGrobol/gists{/gist_id}","starred_url":"https://api.github.com/users/LoicGrobol/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/LoicGrobol/subscriptions","organizations_url":"https://api.github.com/users/LoicGrobol/orgs","repos_url":"https://api.github.com/users/LoicGrobol/repos","events_url":"https://api.github.com/users/LoicGrobol/events{/privacy}","received_events_url":"https://api.github.com/users/LoicGrobol/received_events","type":"User","user_view_type":"public","site_admin":false,"name":"L. Grobol","company":"Université Pa

Le serveur est alors littéralement un serveur web, les concepts s'alignent !

Le fait de passer par HTTP pour faire communiquer des applications a bien des avantages :

- On peut communiquer via Internet (HTTP est prévu pour ça) entre machines très distantes
- On peut bénéficier de toutes les technologies et infrastructures développées pour le web
  (matérielles, logicielles et humaines) qui du fait de leur omniprésence sont très optimisées
- HTTP propose tout un tas d'outils intéressants
  - Les URL avoir des chemins d'accès hiérarchisés
  - Les systèmes d'authentification (sessions, cookies…)
  - La transmission bidirectionnelle de données arbitraires (via les *payloads*)

## REST

_**Re**presentational **s**tate **t**ransfer_ est une méthodologie de conception d'API web, pensée
pour demander le moins de *couplage* possible entre une application client et le serveur à l'API
duquel elle accède. Autrement dit le client n'a besoin que très peu de connaissance du
fonctionnement du serveur et vice-versa.

Les principes (un peu simplifiés) de REST sont

- L'absence de mémoire (*statelessness*) : le serveur ne doit pas garder en mémoire de trace des
  requêtes du client.
- La séparation du client et du serveur : les deux doivent être suffisamment découplés pour pouvoir
  être modifiés sans conséquence de l'un sur l'autre (tant que l'API ne change pas)
- La possibilité de mettre les requêtes en cache (*cacheability*) : les données renvoyées pour une
  requête données doivent être rigoureusement identique d'une requête sur l'autre afin que le client
  comme le serveur puissent les stocker en mémoire cache.
- L'uniformité des interfaces :
  - L'identification des ressources se fait par un identifiant indépendant de leurs représentations
  - La représentation d'une ressource doit être suffisante pour la mettre à jour ou la supprimer du
    serveur
  - Chaque message doit contenir une description de la façon dont il doit être lu
- L'indépendance d'accès (*layered system*) : le comportement de l'interface doit être identique
  quel que soit le moyen utilisé pour y accéder. En particulier, il ne doit pas changer si cet accès
  passe par des *proxies*.

## Accéder à des API

On l'a déjà fait [plusieurs](../01-internet/internets.py.md) [fois](../03-httpx/httpx.py.md) !

On a dit qu'il suffisait de faire des requêtes HTTP et ça on sait déjà faire :

In [8]:
print(httpx.get("https://jsonplaceholder.typicode.com/comments/1").text)

{
  "postId": 1,
  "id": 1,
  "name": "id labore ex et quam laborum",
  "email": "Eliseo@gardner.biz",
  "body": "laudantium enim quasi est quidem magnam voluptate ipsam eos\ntempora quo necessitatibus\ndolor quam autem quasi\nreiciendis et nam sapiente accusantium"
}


Par contre, on a pas reparlé de ce format étrange.

Ça ressemble à la représentation d'un `dict`


In [9]:
import ast
ast.literal_eval(httpx.get("https://jsonplaceholder.typicode.com/comments/1").text)

{'postId': 1,
 'id': 1,
 'name': 'id labore ex et quam laborum',
 'email': 'Eliseo@gardner.biz',
 'body': 'laudantium enim quasi est quidem magnam voluptate ipsam eos\ntempora quo necessitatibus\ndolor quam autem quasi\nreiciendis et nam sapiente accusantium'}

Mais ce n'est pas tout à fait ça

In [10]:
ast.literal_eval(httpx.get("https://jsonplaceholder.typicode.com/todos/1").text)

ValueError: malformed node or string on line 5: <ast.Name object at 0x7d6534dbbb50>

Tiens, d'ailleurs, est-ce que vous voyez le problème ?

In [11]:
print(httpx.get("https://jsonplaceholder.typicode.com/todos/1").text)

{
  "userId": 1,
  "id": 1,
  "title": "delectus aut autem",
  "completed": false
}


## JSON

_**J**ava**S**cript **O**bject **N**otation_. Comme son nom l'indique, c'est (à de tout, tout petits
détails près) la syntaxe pour noter des objets en JavaScript.

C'est très très très proche de la syntaxe des `dict` littéraux en Python. Sauf quand c'est
différent.

Comme d'habitude [MDN](https://developer.mozilla.org/en-US/docs/Learn/JavaScript/Objects/JSON) est
notre meilleur⋅e ami⋅e. Il y a aussi [une description formelle
standard](https://www.rfc-editor.org/info/std90).

Sa (relative) simplicité de lecture et d'écriture en a fait le format privilégié d'échange de
données pour les API web, puis petit à petit aussi le format standard *de facto* pour énormément
d'usages.

C'est facile de le parser en Python et de récupérer un `dict` avec le module natif [json](https://docs.python.org/fr/3/library/json.html)

In [12]:
import json
data_as_a_str = httpx.get("https://jsonplaceholder.typicode.com/todos/1").text
data_as_a_dict = json.loads(data_as_a_str)
data_as_a_dict

{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}

Et la conversion dans l'autre sens n'est pas compliquée non plus

In [13]:
d = {"name": "Launcelot", "quest": "Seek the Holy Grail", "sparrows seen": 2, "fears": [], "married": False, 0: None}
s = json.dumps(d)
s

'{"name": "Launcelot", "quest": "Seek the Holy Grail", "sparrows seen": 2, "fears": [], "married": false, "0": null}'

En plus `httpx` le fait pour nous

In [14]:
data_as_a_dict = httpx.get("https://jsonplaceholder.typicode.com/todos/1").json()
data_as_a_dict

{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}

Même pas besoin de se fatiguer.

Si on veut *envoyer* du JSON, il y a une subtilité :

In [15]:
response = httpx.post(
  "https://jsonplaceholder.typicode.com/todos",
  json={"userId": 1, "title": "Buy milk", "completed": False}
)
response.json()

{'userId': 1, 'title': 'Buy milk', 'completed': False, 'id': 201}

Il faut passer les données au paramètre `json` de `requests.post` et non `data` (ou alors il faut
lui passer sous forme de chaîne de caractère et avoir dans les *headers* `"Content-Type"` qui vaut
`"application/json"`).

Attention, si vous essayez de faire ça dans un `get`, httx ne va pas être d'accord : ce n'est pas une méthode HTTP avec laquelle on est censé envoyer des données.

## 🌐 Exo 🌐

### Le cheeseshop

En utilisant l'[API de PyPI](https://docs.pypi.org/api/json/), écrire un script
qui prend en argument un nom de package et affiche (si un tel package existe) les noms et emails des
auteurices de package et la date de la dernière *release*;

In [29]:
def get_package(package_name):
    try:
        response = httpx.get(f"https://pypi.org/pypi/{package_name}/json", follow_redirects=True)

        if response.status_code == 404:
            print("Ce package n'existe pas.")
            return

        link = response.json()

        print("Le dictionnaire a cette forme :")
        for cle, valeur in link.items():
            print(cle, valeur)

        print("\nOn veut récupérer le nom des auteurices du package :")
        print(link["info"]["author"])

        print("\nMaintenant leur mail :")
        print(link["info"]["author_email"])

        print("\nEt la dernière release :")
        derniere_release = list(link["releases"])[-1]
        print(derniere_release)

        print("\nAvec la date de celle-ci :")
        upload_time = link["releases"][derniere_release][-1]
        print(upload_time["upload_time"])

    except httpx.RequestError as e:
        print(f"Erreur lors de la requête : {e}")

    except Exception as e:
        print(f"Une erreur inattendue s'est produite : {e}")

get_package("prettytable")


Le dictionnaire a cette forme :
info {'author': None, 'author_email': 'Luke Maurits <luke@maurits.id.au>', 'bugtrack_url': None, 'classifiers': ['Programming Language :: Python', 'Programming Language :: Python :: 3 :: Only', 'Programming Language :: Python :: 3.10', 'Programming Language :: Python :: 3.11', 'Programming Language :: Python :: 3.12', 'Programming Language :: Python :: 3.13', 'Programming Language :: Python :: 3.14', 'Programming Language :: Python :: 3.9', 'Programming Language :: Python :: Implementation :: CPython', 'Programming Language :: Python :: Implementation :: PyPy', 'Topic :: Text Processing', 'Typing :: Typed'], 'description': '# PrettyTable\n\n[![PyPI version](https://img.shields.io/pypi/v/prettytable.svg?logo=pypi&logoColor=FFE873)](https://pypi.org/project/prettytable/)\n[![Supported Python versions](https://img.shields.io/pypi/pyversions/prettytable.svg?logo=python&logoColor=FFE873)](https://pypi.org/project/prettytable/)\n[![PyPI downloads](https://img.

### Zenodo

En utilisant l'[API de Zenodo](https://developers.zenodo.org/#rest-api), écrire un script qui prend
en argument un nom de fichier, un titre et un nom d'auteurice ; fait un dépôt sur la sandbox de
Zenodo ; et afficher un lien vers ce nouveau dépôt. Il vous faudra créer un compte pour obtenir un
*access token*.

Mon token d'accès : SfqVjLfEgsgqF5qArwKkCO2gjxFqd4BtAsZMsHRyPhXWiRwJuXK0oLb6W2Y1

In [36]:
!GET /api/deposit/depositions?access_token=SfqVjLfEgsgqF5qArwKkCO2gjxFqd4BtAsZMsHRyPhXWiRwJuXK0oLb6W2Y1

### Philosophie

<!-- LTeX: language=en-GB -->
> Wikipedia trivia: if you take any article, click on the first link in the article text not in
> parentheses or italics, **and then repeat**, you will eventually end up at "Philosophy". ([xkcd
> #903](https://xkcd.com/903/))
<!-- LTeX: language=fr -->

- Vérifiez sur une page ou deux si c'est vrai
- Écrivez un script qui prend en argument de ligne de commande un nom de page Wikipédia (en anglais,
  sauf si vous aimez l'aventure) et donne le nombre de sauts nécessaire pour arriver à la page
  *Philosophy* ou une erreur si la page en question n'existe pas.
  - Utilisez l'[API](https://www.mediawiki.org/wiki/API:Get_the_contents_of_a_page) de Wikipédia
    pour obtenir le contenu des pages.
  - Vous pouvez parser le wikitexte à la main ou utiliser
    [wikitextparser](https://pypi.org/project/wikitextparser/)
- Si vous êtes très déterminé⋅es, faites un script qui prend en entrée des pages de Wikipédia et
  produit le graphe (orienté) des pages obtenues en suivant à chaque fois le premier lien de chaque
  page, et ce jusqu'à retomber sur une page déjà visitée. On pourra par exemple utiliser
  [NetworkX](https://networkx.org/documentation/latest/reference/drawing.html), un visualiseur
  interactif comme [pyvis](https://pyvis.readthedocs.io/en/latest/tutorial.html), [un *wrapper* de
  graphviz](https://graphviz.readthedocs.io) ou encore générer directement des fichiers dot.

